<a href="https://colab.research.google.com/github/eriksonferreira/pattern_recongnition/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.base import BaseEstimator, ClusterMixin, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode


In [45]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:,2:]
y = iris.target

KNN

In [46]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=4)

In [47]:
y_pred_knn = knn.predict(X)

In [48]:
accuracy_knn = accuracy_score(y, y_pred_knn)
accuracy_knn

0.9733333333333334

KMeans

In [49]:
class KMeans(BaseEstimator, ClusterMixin, TransformerMixin):
    def __init__(self, n_clusters=3):
        self.n_clusters = n_clusters

    def _init_centroids(self, X):
        min = np.min(X, axis=0)
        max = np.max(X, axis=0)
        centroids = []
        for i in range(self.n_clusters):
            centroid = []
            for j in range(X.shape[1]):
                centroid.append(random.uniform(min[j], max[j]))
            centroids.append(centroid)
        self.clusters_centers_ = np.array(centroids)

    def fit(self, X, y=None):
        self._init_centroids(X)
        previous_centroids = self.clusters_centers_.copy()
        for _ in range(1):
            ypred = self.predict(X)
            for i in range(self.n_clusters):
                if np.any(ypred == i):
                    self.clusters_centers_[i] = np.mean(X[ypred == i], axis=0)
            if np.allclose(previous_centroids, self.clusters_centers_, atol=0.0001):
                break
            previous_centroids = self.clusters_centers_.copy()

    def _distancias(self, X):
        diferenca = self.clusters_centers_ - X
        quadrado = diferenca ** 2
        soma = np.sum(quadrado, axis=1)
        raiz = np.sqrt(soma)
        return raiz

    def predict(self, X):
        ypred = np.empty((X.shape[0],))
        for i in range(X.shape[0]):
            distancias = self._distancias(X[i])
            centroid = np.argmin(distancias)
            ypred[i] = centroid
        return ypred

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [51]:
kmeans = KMeans()
kmeans.fit(X_train)

In [52]:
y_pred_kmeans_test = kmeans.predict(X_test)

In [53]:
labels = np.zeros_like(y_pred_kmeans_test)
for i in range(kmeans.n_clusters):
    mask = (kmeans.predict(X_train) == i)
    labels_train = y_train[mask]
    # Encontrar a moda das etiquetas reais correspondentes aos clusters
    # print(mode(labels_train)[0])
    most_common = mode(labels_train)[0]
    labels[y_pred_kmeans_test == i] = most_common

In [54]:
accuracy = accuracy_score(y_test, labels)
accuracy

0.9333333333333333